# Caso Práctico (tips) - ENGIE
*Jose Enrique Zafra Mena*

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import json 

## Carga de los datos

In [106]:
facturas_df = pd.read_excel('datos/datos-tips/AGIkey_facturas.xlsx')
mapping_df = pd.read_excel('datos/Mapping.xlsx')
IAC_df = pd.read_excel('datos/datos-tips/MEDG2_IAC_GETRA_ABRIL_2024_06_18_13_34.xlsx')
CTV_df = pd.read_excel('datos/datos-tips/JVLNG_CTV_STOK_ABRIL_2024_06_18_13_34.xlsx')

In [107]:
facturas_df.head(100)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,...,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones,Importe (<0 a pagar EEE-Enagas),mes,año
0,1400180000,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"4,56",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
1,1400180001,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"15794,20",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
2,1400180002,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"16380,50",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
3,1400180003,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"1930,22",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
4,1400180004,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"43388,45",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2024262664,ESA86484292,"ENAGAS GTS, SAU",60.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,2024-05-02 00:00:00,"128922,44",EUR,...,2024-05-02 00:00:00,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN,NaN,NaN,NaN
96,2024262665,ESA86484292,"ENAGAS GTS, SAU",60.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,2024-05-02 00:00:00,"136751,51",EUR,...,2024-05-02 00:00:00,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN,NaN,NaN,NaN
97,2024262666,ESA86484292,"ENAGAS GTS, SAU",60.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,2024-05-02 00:00:00,"442,51",EUR,...,2024-05-02 00:00:00,eMail,344991,AVB,Entrada PVB desde AVB,NaN,NaN,NaN,NaN,NaN
98,2024262667,ESA86484292,"ENAGAS GTS, SAU",60.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,2024-05-02 00:00:00,"260,02",EUR,...,2024-05-02 00:00:00,eMail,344991,AVB,Entrada PVB desde AVB,NaN,NaN,NaN,NaN,NaN


## Primera parte

### Filtrado de datos relevantes

#### Facturas
Solo nos interesan ciertas columnas: Factura, Importe, Fecha Factura.

Además, cambiamos el formato de la fecha para poder filtrar por Abril.

Salen *warnings*, pero no tienen relevancia.

In [108]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen','ServicioFacturado']]

# solo queremos algunos valores de Origen: C.I.Almería y Entrada PVB
facturas_filtrado_df = facturas_df[(facturas_df['Origen'].isin(['C.I. Almería'])) & (facturas_df['ServicioFacturado'].isin(['Entrada PVB']))]

# cambiamos el formato del importe
facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',', '.').astype(float)

C:\Users\kikek\AppData\Local\Temp\ipykernel_33588\2745930555.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado_df['Importe'] = facturas_filtrado_df['Importe'].str.replace(',', '.').astype(float)


In [109]:
# cambiamos el formato de la fecha
facturas_filtrado_df['FechaFactura'] = pd.to_datetime(facturas_filtrado_df['FechaFactura'], utc = True)

facturas_filtrado_df.head(10)

# filtramos para abril del 24
facturas_filtrado_df = facturas_filtrado_df[(facturas_filtrado_df['FechaFactura'].dt.month == 4) & (facturas_filtrado_df['FechaFactura'].dt.year == 2024)]

facturas_filtrado_df.head(10)

C:\Users\kikek\AppData\Local\Temp\ipykernel_33588\2206809554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado_df['FechaFactura'] = pd.to_datetime(facturas_filtrado_df['FechaFactura'], utc = True)


,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
570,2324005598,536445.62,2024-04-04 00:00:00+00:00,C.I. Almería,Entrada PVB


#### Mapping

In [110]:
mapping_df = mapping_df.drop_duplicates()
print(mapping_df.shape)

# solo nos interesan algunos valores: Commodity == IAC
mapping_filtrado_df = mapping_df[(mapping_df['Commodity'].isin(['IAC']))]
mapping_filtrado_df.head(200)

(29, 5)


,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


#### Deals 
Solo queremos las columnas: Deal, Cantidad, Start-DeliveryDate, End-DeliveryDate, Procedencia (SUGST_AOC_STOK), Subflow 1, Date 1, Subcantidad 1, Subflow 2, Date 2, Subcantidad 2.

En la columna flows vemos que hay datos en json. Tratamos esa columna para sacar los datos a un formato mejor. También mantemos el Id del deal para cada subflow.


In [111]:
IAC_df = IAC_df.drop_duplicates()

# nos quedamos con las columnas que queremos por ahora
IAC_filtrado_df = IAC_df[['Id', 'Flows', 'StartDeliveryDate', 'EndDeliveryDate']]
IAC_filtrado_df = IAC_filtrado_df.rename(columns={'Id':'Id Deal'})

IAC_filtrado_df.head()

,Id Deal,Flows,StartDeliveryDate,EndDeliveryDate
0,37591650,"[{'Currency': 'EUR', 'Amount': 105279.586144, ...",2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00
1,37056084,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 9219...",2023-01-01T00:00:00+01:00,2023-12-31T00:00:00+01:00


In [112]:
# definimos una función para tratar los json (la tendremos que usar varias veces)
def clean_json(string):
    if isinstance(string, str):
        string = string.replace("'", '"')
        string = string.replace("None", 'null')
        string = string.replace("True", 'true')
        string = string.replace("False", 'false')
    return string

# la usamos sobre la columna de flows
flows_cleaned = IAC_filtrado_df['Flows'].apply(clean_json)


# Ahora creamos otra función para extraer los datos de json a diccionarios de python
def extract_json(flows, id_deal):
    if not flows or not isinstance(flows, list):
        return pd.DataFrame()
    data = {
        'Id Deal': [],
        'Currency': [],
        'Amount': [],
        'Id': [],
        'Date': [],
        'FlowType': [],
        'PayReceive': []
    }
    for item in flows:
        if isinstance(item, dict):
            data['Id Deal'].append(id_deal)
            for key in data.keys():
                if key != 'Id Deal':
                    data[key].append(item.get(key))
        else:
            print(f"Unexpected type {type(item)}")
    return pd.DataFrame(data)

# creamos un df para meter los datos
datos_json = []

for idx, row in IAC_filtrado_df.iterrows():
    try:
        # intentamos convertir la cadena a una lista de diccionarios
        print(f"Procesamos fila {idx} con Id Deal {row['Id Deal']}")
        flows = eval(row['Flows'])
        id_deal = row['Id Deal']
        i = extract_json(flows, id_deal)
        if not i.empty:
            datos_json.append(i)
        else:
            print(f"DataFrame vacío para Id Deal {id_deal}")
    except Exception as e:
        print(e)

datos_json_final = pd.concat(datos_json, ignore_index=True)
    
# cambiamos un poco las columnas
datos_json_final = datos_json_final[['Id Deal','Id','Amount', 'Date', 'FlowType']]
datos_json_final = datos_json_final.rename(columns={'Id':'Id Flow', 'Amount':'Importe Flow', 'Date':'Fecha Flow', 'FlowType':'Subflow'})

datos_json_final.head(100)

Procesamos fila 0 con Id Deal 37591650
Procesamos fila 1 con Id Deal 37056084


,Id Deal,Id Flow,Importe Flow,Fecha Flow,Subflow
0,37591650,93540225,105279.586144,2024-02-20T00:00:00+01:00,1
1,37591650,93540224,409582.490000,2024-02-20T00:00:00+01:00,2
2,37591650,93540227,120318.957648,2024-03-20T00:00:00+01:00,1
3,37591650,93540226,383444.510000,2024-03-20T00:00:00+01:00,2
4,37591650,93540229,128084.859614,2024-04-22T00:00:00+02:00,1
5,37591650,93540228,408200.220000,2024-04-22T00:00:00+02:00,2
6,37591650,93540231,130004.564430,2024-05-20T00:00:00+02:00,1
7,37591650,93540230,395032.480000,2024-05-20T00:00:00+02:00,2
8,37591650,93540233,134338.049911,2024-06-20T00:00:00+02:00,1
9,37591650,93540232,408200.220000,2024-06-20T00:00:00+02:00,2


Lo anterior solo es la columna que teníamos de flows. Ahora lo unimos con el resto del dataframe original (deals IAC). Tan solo tenemos que añadirle las columnas de las fechas iniciales y finales, según el valor de Id Deal que tenga.

También cambiamos el formato de las fechas y filtramos para el mes de Abril

In [145]:
IAC_final_df = (pd.merge(IAC_filtrado_df, datos_json_final, on='Id Deal', how='inner')).drop_duplicates()
IAC_final_df = IAC_final_df.drop(columns=['Flows'])

# también añadimos una columna llamada Commodity con valor IAC para que sea más fácil el merge
IAC_final_df['Commodity']='IAC'

# cambiamos el formato de las fechas (por lo menos el de Fecha Flow)
IAC_final_df['Fecha Flow'] = pd.to_datetime(IAC_final_df['Fecha Flow'], utc=True)

In [113]:

# filtramos para el mes de Abril
IAC_final_df = IAC_final_df[IAC_final_df['Fecha Flow'].dt.month == 4]
IAC_final_df = IAC_final_df[IAC_final_df['Fecha Flow'].dt.year == 2024]

IAC_final_df.head(100)


,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity
4,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540229,128084.859614,2024-04-21 22:00:00+00:00,1,IAC
5,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540228,408200.220000,2024-04-21 22:00:00+00:00,2,IAC


### Merge de Mapping con Deals
Hacemos el merge con Commodity

In [114]:
IAC_merged_df = pd.merge(IAC_final_df, mapping_filtrado_df, on='Commodity', how='inner')

# quitamos algunas columnas innecesarias
IAC_merged_df.drop(columns=['Portfolio','DealType'], inplace=True)


IAC_merged_df.head(50)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity,Origen,Servicio facturado
0,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540229,128084.859614,2024-04-21 22:00:00+00:00,1,IAC,C.I. Almería,Entrada PVB
1,37591650,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,93540228,408200.220000,2024-04-21 22:00:00+00:00,2,IAC,C.I. Almería,Entrada PVB


### Merge de Mapping con Facturas
En este caso nos vale bien con Origen, o bien con Servicio Facturado.

In [115]:
facturas_merged_df = pd.merge(facturas_filtrado_df, mapping_filtrado_df, on='Origen', how='inner')


facturas_merged_df = facturas_merged_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged_df = facturas_merged_df.loc[:, ~facturas_merged_df.columns.duplicated()]

facturas_merged_df.head(100)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,2324005598,536445.62,2024-04-04 00:00:00+00:00,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


### Conciliación
**Se hace según *Origen* + *Servicio Facturado***

Mapping y Facturas comparten *Origen* y *Servicio Facturado*

Vemos la cantidad según Facturas, la cantidad según Deals, y hacemos la diferencia

#### Comparación entre Facturas y Deals


In [116]:
# agrupamos y sumamos importes
IAC_grouped = IAC_merged_df.groupby(['Id Deal','Fecha Flow','Origen', 'Servicio facturado', 'Subflow'])['Importe Flow'].sum()

IAC_grouped.head(100)

Id Deal   Fecha Flow                 Origen        Servicio facturado  Subflow
37591650  2024-04-21 22:00:00+00:00  C.I. Almería  Entrada PVB         1          128084.859614
                                                                       2          408200.220000
Name: Importe Flow, dtype: float64

In [117]:
# TODO: usar columnas extras del Excel inicial de Facturas (validación BO, Conciliado, ...) (quizas solo para el df de conciliación)


# hacemos lo mismo para facturas
facturas_grouped = facturas_merged_df.groupby(['NumeroFactura','FechaFactura','Origen', 'Servicio facturado'])['Importe'].sum()

facturas_grouped.head(100)

NumeroFactura  FechaFactura               Origen        Servicio facturado
2324005598     2024-04-04 00:00:00+00:00  C.I. Almería  Entrada PVB           536445.62
Name: Importe, dtype: float64

#### DataFrame de conciliación
Falta poner columnas de Subflows, Numero de Factura, Id Deal

In [118]:
# creamos un dataframe solo con el importe total de los deals
IAC_total_df = IAC_merged_df.groupby(['Id Deal','Origen', 'Servicio facturado'])['Importe Flow'].sum()

# unimos los dataframes para comparar
conciliacion_df = pd.merge(IAC_total_df, facturas_grouped, on=['Origen', 'Servicio facturado'], how='outer')

# hallamos la diferencia 
conciliacion_df['Factura-Deal'] = conciliacion_df['Importe'] - conciliacion_df['Importe Flow']  

conciliacion_df.head()

,,Importe Flow,Importe,Factura-Deal
Origen,Servicio facturado,,,
C.I. Almería,Entrada PVB,536285.079614,536445.62,160.540386


## Segunda Parte
Seguiremos el mismo procedimiento que antes.

### Filtrado de datos relevantes

#### Facturas
Solo queremos Origen == TVB y Servicio Facturado == Almacenamiento TVB.

Filtrar para mes de marzo, fecha factura Abril 2024

In [119]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen', 'ServicioFacturado']]

# filtramos los valores que queremos
facturas_filtrado2_df = facturas_df[(facturas_df['Origen'].isin(['TVB'])) & (facturas_df['ServicioFacturado'].isin(['Almacenamiento TVB']))]

# cambiamos el formato del importe
facturas_filtrado2_df['Importe'] = facturas_filtrado2_df['Importe'].str.replace(',', '.').astype(float)

# cambiamos el formato de la fecha
facturas_filtrado2_df['FechaFactura'] = pd.to_datetime(facturas_filtrado2_df['FechaFactura'], utc=True)

# filtramos para abril del 24
facturas_filtrado2_df = facturas_filtrado2_df[(facturas_filtrado2_df['FechaFactura'].dt.month == 4) & (facturas_filtrado2_df['FechaFactura'].dt.year == 2024)]

facturas_filtrado2_df.head(100)

C:\Users\kikek\AppData\Local\Temp\ipykernel_33588\2108121968.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado2_df['Importe'] = facturas_filtrado2_df['Importe'].str.replace(',', '.').astype(float)
C:\Users\kikek\AppData\Local\Temp\ipykernel_33588\2108121968.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_filtrado2_df['FechaFactura'] = pd.to_datetime(facturas_filtrado2_df['FechaFactura'], utc=True)


,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
279,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
280,2024164583,718.60,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
281,2024164584,1015.95,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
282,2024164585,1017.59,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
283,2024164586,1030.97,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
284,2024164587,1041.67,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
285,2024164588,1965.66,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
286,2024164589,1472.13,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
287,2024164590,1930.23,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB
288,2024164591,1787.43,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB


#### Mapping
Ahora tenemos que filtrar para Commodity == CTV, y Portfolio == JVLNG.

Realmente creo que esto se podría optimizar solo con el paso del merge, pero ahora no se me ocurre cómo.

In [120]:
mapping_df = mapping_df.drop_duplicates()
mapping_filtrado2_df = mapping_df[(mapping_df['Commodity'].isin(['CTV'])) & (mapping_df['Portfolio'].isin(['JVLNG']))]

mapping_filtrado2_df.head(100)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
16,TVB,Almacenamiento TVB,JVLNG,CTV,STOK


#### Deals
Se supone que ya está filtrado en el deal (la fecha??). Hay que localizar los subflows.

In [121]:
CTV_df.head()

,Unnamed: 0,AggregatedKey,ApplicationPeriod,BookingSource,BOValidated,BOValidationDate,BOValidator,CancelsDealId,Capacity,CapacityFrequency,...,ExerciseDate,ExerciseType,Nature,OptionNature,OptionType,PeriodInfoCollection,Script,TemplateId,TemplateScript,OrderId
0,0,"42965422,2024/06/13 13:03:21,GEFWD",NONSD,MET,False,1900-01-01T00:00:00+01:00,False,0,15000.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"42563586,2024/05/20 13:35:59,GEFWD",010524,MET,True,2024-05-20T13:35:59+02:00,GGH410,0,90000.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"42518127,2024/05/20 13:34:13,GEFWD",040624,MET,True,2024-05-20T13:34:13+02:00,GGH410,0,50000.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"42518126,2024/05/20 13:35:59,GEFWD",010624,MET,True,2024-05-20T13:35:59+02:00,GGH410,0,50000.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"42518125,2024/05/20 13:35:59,GEFWD",010524,MET,True,2024-05-20T13:35:59+02:00,GGH410,0,50000.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
CTV_df = CTV_df.drop_duplicates()

# nos quedamos con las columnas que queremos por ahora
CTV_filtrado_df = CTV_df[['Id', 'Flows', 'StartDeliveryDate', 'EndDeliveryDate']]
CTV_filtrado_df = CTV_filtrado_df.rename(columns={'Id':'Id Deal'})

CTV_filtrado_df.head()

,Id Deal,Flows,StartDeliveryDate,EndDeliveryDate
0,42965422,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1093...",2024-06-15T00:00:00+02:00,2024-06-24T00:00:00+02:00
1,42563586,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1083...",2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00
2,42518127,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1082...",2024-06-04T00:00:00+02:00,2024-06-04T00:00:00+02:00
3,42518126,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1082...",2024-06-01T00:00:00+02:00,2024-06-01T00:00:00+02:00
4,42518125,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1082...",2024-05-01T00:00:00+02:00,2024-05-01T00:00:00+02:00


In [123]:
# usamos las funciones definidas en la primera parte
flows2_cleaned = CTV_filtrado_df['Flows'].apply(clean_json)

# creamos un df para meter los datos
datos2_json = []

for idx, row in CTV_filtrado_df.iterrows():
    try:
        # intentamos convertir la cadena a una lista de diccionarios
        flows = eval(row['Flows'])
        id_deal = row['Id Deal']
        i = extract_json(flows, id_deal)
        if not i.empty:
            datos2_json.append(i)
        else:
            print(f"DataFrame vacío para Id Deal {id_deal}")
    except Exception as e:
        print(e)

datos2_json_final = pd.concat(datos2_json, ignore_index=True)
    
# cambiamos un poco las columnas
datos2_json_final = datos2_json_final[['Id Deal','Id','Amount', 'Date', 'FlowType']]
datos2_json_final = datos2_json_final.rename(columns={'Id':'Id Flow', 'Amount':'Importe Flow', 'Date':'Fecha Flow', 'FlowType':'Subflow'})

datos2_json_final.head(1000)

'{' was never closed (<string>, line 1)
unterminated string literal (detected at line 1) (<string>, line 1)
unterminated string literal (detected at line 1) (<string>, line 1)


,Id Deal,Id Flow,Importe Flow,Fecha Flow,Subflow
0,42965422,109381457,0.0,2024-07-22T00:00:00+02:00,1
1,42965422,109381458,0.0,2024-07-22T00:00:00+02:00,1
2,42965422,109381459,0.0,2024-07-22T00:00:00+02:00,1
3,42965422,109381460,0.0,2024-07-22T00:00:00+02:00,1
4,42965422,109381461,0.0,2024-07-22T00:00:00+02:00,1
...,...,...,...,...,...
320,31915597,77563057,0.0,2022-11-18T00:00:00+01:00,1
321,31915597,77563058,0.0,2022-12-20T00:00:00+01:00,2
322,31915597,77563059,0.0,2022-12-20T00:00:00+01:00,1
323,31915597,77563060,0.0,2023-01-20T00:00:00+01:00,2


Proseguimos como en la primera parte,

In [124]:
CTV_final_df = (pd.merge(CTV_filtrado_df, datos2_json_final, on='Id Deal', how='inner')).drop_duplicates()
CTV_final_df = CTV_final_df.drop(columns=['Flows'])

# también añadimos una columna llamada Commodity con valor CTV para que sea más fácil el merge
CTV_final_df['Commodity']='CTV'

# cambiamos el formato de las fechas (por lo menos el de Fecha Flow)
CTV_final_df['Fecha Flow'] = pd.to_datetime(CTV_final_df['Fecha Flow'], utc=True)

# filtramos para el mes de Abril
CTV_final_df = CTV_final_df[CTV_final_df['Fecha Flow'].dt.month == 4]
CTV_final_df = CTV_final_df[CTV_final_df['Fecha Flow'].dt.year == 2024]

CTV_final_df.head(100)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity
36,41985669,2024-03-15T00:00:00+01:00,2024-03-15T00:00:00+01:00,106810287,0.0,2024-04-01 22:00:00+00:00,1,CTV
41,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950849,0.0,2024-04-18 22:00:00+00:00,2,CTV
42,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950850,0.0,2024-04-18 22:00:00+00:00,1,CTV
139,37761994,2022-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,94022898,0.0,2024-04-18 22:00:00+00:00,2,CTV
140,37761994,2022-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,94022899,0.0,2024-04-18 22:00:00+00:00,1,CTV
165,37737643,2023-12-31T00:00:00+01:00,2025-12-31T00:00:00+01:00,93939943,0.0,2024-04-18 22:00:00+00:00,2,CTV
166,37737643,2023-12-31T00:00:00+01:00,2025-12-31T00:00:00+01:00,93939944,0.0,2024-04-18 22:00:00+00:00,1,CTV
215,37734719,2023-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,93935335,0.0,2024-04-18 22:00:00+00:00,2,CTV
216,37734719,2023-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,93935336,0.0,2024-04-18 22:00:00+00:00,1,CTV


### Merge de Mapping con Deals
Hacemos el merge con Commodity

In [125]:
CTV_merged_df = pd.merge(CTV_final_df, mapping_filtrado2_df, on='Commodity', how='inner')

# quitamos algunas columnas innecesarias
CTV_merged_df.drop(columns=['Portfolio','DealType'], inplace=True)

CTV_merged_df.head(10)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity,Origen,Servicio facturado
0,41985669,2024-03-15T00:00:00+01:00,2024-03-15T00:00:00+01:00,106810287,0.0,2024-04-01 22:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
1,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950849,0.0,2024-04-18 22:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
2,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950850,0.0,2024-04-18 22:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
3,37761994,2022-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,94022898,0.0,2024-04-18 22:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
4,37761994,2022-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,94022899,0.0,2024-04-18 22:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
5,37737643,2023-12-31T00:00:00+01:00,2025-12-31T00:00:00+01:00,93939943,0.0,2024-04-18 22:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
6,37737643,2023-12-31T00:00:00+01:00,2025-12-31T00:00:00+01:00,93939944,0.0,2024-04-18 22:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
7,37734719,2023-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,93935335,0.0,2024-04-18 22:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
8,37734719,2023-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,93935336,0.0,2024-04-18 22:00:00+00:00,1,CTV,TVB,Almacenamiento TVB


### Merge de Mapping con Facturas

In [126]:
facturas_merged2_df = pd.merge(facturas_filtrado2_df, mapping_df, on='Origen', how='inner')

facturas_merged2_df = facturas_merged2_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged2_df = facturas_merged2_df.loc[:, ~facturas_merged2_df.columns.duplicated()]


facturas_merged2_df.head(100)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
1,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,CTV,STOK
2,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
3,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
4,2024164582,59080.41,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2F,GETRA
...,...,...,...,...,...,...,...,...
61,2024164592,1375.12,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,CTV,STOK
62,2024164592,1375.12,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
63,2024164592,1375.12,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
64,2024164592,1375.12,2024-04-03 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2F,GETRA


### Conciliación

#### Comparación entre Facturas y Deals

In [127]:
# agrupamos y sumamos importes
CTV_grouped = CTV_merged_df.groupby(['Id Deal', 'Fecha Flow', 'Origen', 'Servicio facturado', 'Subflow'])[
    'Importe Flow'].sum()

# TODO: añadir columna de Id Subflow (está en el merge)

CTV_grouped.head(100)

Id Deal   Fecha Flow                 Origen  Servicio facturado  Subflow
37734719  2024-04-18 22:00:00+00:00  TVB     Almacenamiento TVB  1          0.0
                                                                 2          0.0
37737643  2024-04-18 22:00:00+00:00  TVB     Almacenamiento TVB  1          0.0
                                                                 2          0.0
37761994  2024-04-18 22:00:00+00:00  TVB     Almacenamiento TVB  1          0.0
                                                                 2          0.0
39942100  2024-04-18 22:00:00+00:00  TVB     Almacenamiento TVB  1          0.0
                                                                 2          0.0
41985669  2024-04-01 22:00:00+00:00  TVB     Almacenamiento TVB  1          0.0
Name: Importe Flow, dtype: float64

In [128]:
# TODO: usar columnas extras del Excel inicial de Facturas (validación BO, Conciliado, ...)


# hacemos lo mismo para facturas
facturas2_grouped = facturas_merged2_df.groupby(['NumeroFactura', 'FechaFactura', 'Origen', 'Servicio facturado'])[
    'Importe'].sum()

facturas2_grouped.head(100)

NumeroFactura  FechaFactura               Origen  Servicio facturado
2024164582     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB    354482.46
2024164583     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      4311.60
2024164584     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      6095.70
2024164585     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      6105.54
2024164586     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      6185.82
2024164587     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      6250.02
2024164588     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB     11793.96
2024164589     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      8832.78
2024164590     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB     11581.38
2024164591     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB     10724.58
2024164592     2024-04-03 00:00:00+00:00  TVB     Almacenamiento TVB      8250.72
Name: Importe, dtype: float64

## Tips para la Segunda Parte
Tenemos el problema de que **todos los importes de 2024 de los *Subflows* son nulos** (de los Deals proporcionados anteriormente). 

En los nuevos *tips* tenemos las Facturas de Marzo, **pero realmente están filtradas en Febrero**. También tenemos más Deals. Proseguimos como sigue:

1. Sacamos los nuevos Subflows de los Deals ahora incluidos.
2. Filtramos los Subflows por fecha (qué mes??).
3. Hacemos de nuevo el merge del Mapping con los Deals.
4. Estudiamos la nueva conciliación.


También tenemos que lidiar con algunos problemas: 
- **¿Para qué mes hay que filtrar las facturas?** En los últimos tips están filtrados en Febrero, aunque en su nombre pone Marzo; y en los primeros tips se pide: "Mes de Marzo, fecha factura Abril 2024" (incoherente). **Usaré Febrero para comparar con los últimos *tips***
- ¿Para qué mes hay que filtrar los Subflows? Voy a *suponer* Abril, que es lo que pone en el título del Excel de Deals.

### Extracción de los nuevos Deals

In [131]:
CTV_new_df = pd.read_csv("datos/datos-tips/TRS_JVLNG_CTV_STOK_EXTRACTION.csv")

# nos quedamos con las columnas que queremos por ahora
CTV_new_df = CTV_new_df[['Id', 'Flows', 'StartDeliveryDate', 'EndDeliveryDate']]
CTV_new_df = CTV_new_df.rename(columns={'Id':'Id Deal'})

CTV_new_df.head()

,Id Deal,Flows,StartDeliveryDate,EndDeliveryDate
0,43019362,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1095...",2024-09-09T00:00:00+02:00,2024-09-09T00:00:00+02:00
1,43019359,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1095...",2024-09-14T00:00:00+02:00,2024-09-14T00:00:00+02:00
2,43016083,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1095...",2024-09-14T00:00:00+02:00,2024-09-14T00:00:00+02:00
3,43016079,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1095...",2024-09-09T00:00:00+02:00,2024-09-09T00:00:00+02:00
4,42965422,"[{'Currency': 'EUR', 'Amount': 0.0, 'Id': 1093...",2024-06-15T00:00:00+02:00,2024-06-24T00:00:00+02:00


### Extracción de los Subflows


In [135]:
# usamos las funciones definidas en la primera parte
flows3_cleaned = CTV_new_df['Flows'].apply(clean_json)

# creamos un df para meter los datos
datos3_json = []

for idx, row in CTV_new_df.iterrows():
    try:
        # intentamos convertir la cadena a una lista de diccionarios
        flows = eval(row['Flows'])
        id_deal = row['Id Deal']
        i = extract_json(flows, id_deal)
        if not i.empty:
            datos3_json.append(i)
        else:
            print(f"DataFrame vacío para Id Deal {id_deal}")
    except Exception as e:
        print(e)

datos3_json_final = pd.concat(datos3_json, ignore_index=True)
    
# cambiamos un poco las columnas
datos3_json_final = datos3_json_final[['Id Deal','Id','Amount', 'Date', 'FlowType']]
datos3_json_final = datos3_json_final.rename(columns={'Id':'Id Flow', 'Amount':'Importe Flow', 'Date':'Fecha Flow', 'FlowType':'Subflow'})

datos3_json_final.head(10000)

,Id Deal,Id Flow,Importe Flow,Fecha Flow,Subflow
0,43019362,109520371,0.0,2024-10-21T00:00:00+02:00,1
1,43019359,109520368,0.0,2024-10-21T00:00:00+02:00,1
2,43016083,109512985,0.0,2024-10-21T00:00:00+02:00,1
3,43016079,109512896,0.0,2024-10-21T00:00:00+02:00,1
4,42965422,109381457,0.0,2024-07-22T00:00:00+02:00,1
...,...,...,...,...,...
1062,31915597,77563057,0.0,2022-11-18T00:00:00+01:00,1
1063,31915597,77563058,0.0,2022-12-20T00:00:00+01:00,2
1064,31915597,77563059,0.0,2022-12-20T00:00:00+01:00,1
1065,31915597,77563060,0.0,2023-01-20T00:00:00+01:00,2


Ahora tenemos más de mil Subflows. Podemos ver además, que **todos los subflows 1 tienen importe nulo**. Los únicos con importe mayor que cero son subflow 2.

### Filtrado de los Subflows por fecha
 Filtramos por Febrero,

In [143]:
CTV_Feb_df = (pd.merge(CTV_new_df, datos3_json_final, on='Id Deal', how='inner')).drop_duplicates()
CTV_Feb_df = CTV_Feb_df.drop(columns=['Flows'])

# también añadimos una columna llamada Commodity con valor CTV para que sea más fácil el merge
CTV_Feb_df['Commodity']='CTV'

# cambiamos el formato de las fechas (por lo menos el de Fecha Flow)
CTV_Feb_df['Fecha Flow'] = pd.to_datetime(CTV_Feb_df['Fecha Flow'], utc=True)

# filtramos para el mes de Abril
CTV_Feb_df = CTV_Feb_df[CTV_Feb_df['Fecha Flow'].dt.month == 2]
CTV_Feb_df = CTV_Feb_df[CTV_Feb_df['Fecha Flow'].dt.year == 2024]

CTV_Feb_df.head(100)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity
47,39980344,2023-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,101112344,0.000,2024-02-19 23:00:00+00:00,1,CTV
48,39980344,2023-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,101112343,64092.160,2024-02-19 23:00:00+00:00,2,CTV
113,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950845,0.000,2024-02-19 23:00:00+00:00,2,CTV
114,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950846,0.000,2024-02-19 23:00:00+00:00,1,CTV
186,39057923,NaN,NaN,99583560,0.001,2024-02-06 23:00:00+00:00,2,CTV
187,39057923,NaN,NaN,99583561,0.001,2024-02-29 23:00:00+00:00,2,CTV
388,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307303,297308.000,2024-02-06 23:00:00+00:00,2,CTV
389,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307304,297308.000,2024-02-29 23:00:00+00:00,2,CTV
587,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307346,0.000,2024-02-19 23:00:00+00:00,1,CTV
719,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307370,0.000,2024-02-19 23:00:00+00:00,1,CTV


Ahora tenemos 3 subflows con importe distinto de cero (los que valen 0.001 también los considero nulos).

### Filtrado de Facturas por fecha
Como hemos comentado antes, filtramos para Febrero para poder comparar resultados

In [142]:
# filtramos los valores que queremos
facturas_Feb_df = facturas_df[(facturas_df['Origen'].isin(['TVB'])) & (facturas_df['ServicioFacturado'].isin(['Almacenamiento TVB']))]

# cambiamos el formato del importe
facturas_Feb_df['Importe'] = facturas_Feb_df['Importe'].str.replace(',', '.').astype(float)

# cambiamos el formato de la fecha
facturas_Feb_df['FechaFactura'] = pd.to_datetime(facturas_Feb_df['FechaFactura'], utc=True)

# filtramos para FEBRERO del 24
facturas_Feb_df = facturas_Feb_df[(facturas_Feb_df['FechaFactura'].dt.month == 2) & (facturas_Feb_df['FechaFactura'].dt.year == 2024)]

facturas_Feb_df.head(100)

C:\Users\kikek\AppData\Local\Temp\ipykernel_33588\137881255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_Feb_df['Importe'] = facturas_Feb_df['Importe'].str.replace(',', '.').astype(float)
C:\Users\kikek\AppData\Local\Temp\ipykernel_33588\137881255.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facturas_Feb_df['FechaFactura'] = pd.to_datetime(facturas_Feb_df['FechaFactura'], utc=True)


,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
52,2024162757,64092.16,2024-02-02 00:00:00+00:00,TVB,Almacenamiento TVB
505,2024166293,76168.59,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
506,2024166294,825.01,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
507,2024166295,785.84,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
508,2024166296,766.26,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
509,2024166297,805.43,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
510,2024166298,746.68,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
511,2024166299,312.89,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
512,2024166300,723.60,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB
513,2024166301,293.32,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB


Tenemos los mismos valores que los dados en los tips, excepto por una factura, la que tiene NumeroFactura = 2024162757 (con importe 64092.16). También difiere en fecha con el resto (en vez del 4 de febrero, es del 2). No es un error, ya que el importe coincide exactamente con el importe de un Subflow (Id Flow = 101112343).

### Merge de Mapping con Deals


In [147]:
CTV_new_merged_df = pd.merge(CTV_Feb_df, mapping_filtrado2_df, on='Commodity', how='inner')

# quitamos algunas columnas innecesarias
CTV_new_merged_df.drop(columns=['Portfolio','DealType'], inplace=True)

CTV_new_merged_df.head(100)

,Id Deal,StartDeliveryDate,EndDeliveryDate,Id Flow,Importe Flow,Fecha Flow,Subflow,Commodity,Origen,Servicio facturado
0,39980344,2023-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,101112344,0.000,2024-02-19 23:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
1,39980344,2023-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,101112343,64092.160,2024-02-19 23:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
2,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950845,0.000,2024-02-19 23:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
3,39942100,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,100950846,0.000,2024-02-19 23:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
4,39057923,NaN,NaN,99583560,0.001,2024-02-06 23:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
5,39057923,NaN,NaN,99583561,0.001,2024-02-29 23:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
6,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307303,297308.000,2024-02-06 23:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
7,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307304,297308.000,2024-02-29 23:00:00+00:00,2,CTV,TVB,Almacenamiento TVB
8,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307346,0.000,2024-02-19 23:00:00+00:00,1,CTV,TVB,Almacenamiento TVB
9,39051838,2024-01-01T00:00:00+01:00,2034-12-31T00:00:00+01:00,98307370,0.000,2024-02-19 23:00:00+00:00,1,CTV,TVB,Almacenamiento TVB


 ### Merge de Mapping con Facturas

In [149]:
facturas_merged3_df = pd.merge(facturas_Feb_df, mapping_df, on='Origen', how='inner')

facturas_merged3_df = facturas_merged3_df.rename(columns={'ServicioFacturado':'Servicio facturado'})
facturas_merged3_df = facturas_merged3_df.loc[:, ~facturas_merged3_df.columns.duplicated()]


facturas_merged3_df.head(1000)

,NumeroFactura,Importe,FechaFactura,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,2024162757,64092.16,2024-02-02 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,ICC,GETRA
1,2024162757,64092.16,2024-02-02 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,CTV,STOK
2,2024162757,64092.16,2024-02-02 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
3,2024162757,64092.16,2024-02-02 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
4,2024162757,64092.16,2024-02-02 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2F,GETRA
...,...,...,...,...,...,...,...,...
211,2024166327,58.63,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,CTV,STOK
212,2024166327,58.63,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
213,2024166327,58.63,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2C,GETRA
214,2024166327,58.63,2024-02-04 00:00:00+00:00,TVB,Almacenamiento TVB,JVLNG,I2F,GETRA


### Nueva Conciliación

#### Deals

In [156]:
# agrupamos y sumamos importes
CTV_new_grouped = CTV_new_merged_df.groupby(['Id Deal', 'Fecha Flow', 'Origen', 'Servicio facturado','Id Flow', 'Subflow'])[
    'Importe Flow'].sum()


CTV_new_grouped.head(100)

Id Deal   Fecha Flow                 Origen  Servicio facturado  Id Flow    Subflow
34835329  2024-02-19 23:00:00+00:00  TVB     Almacenamiento TVB  89486275   2               0.000
                                                                 89486276   1               0.000
37734719  2024-02-19 23:00:00+00:00  TVB     Almacenamiento TVB  93935331   2               0.000
                                                                 93935332   1               0.000
37737643  2024-02-19 23:00:00+00:00  TVB     Almacenamiento TVB  93939939   2               0.000
                                                                 93939940   1               0.000
37761994  2024-02-19 23:00:00+00:00  TVB     Almacenamiento TVB  94022894   2               0.000
                                                                 94022895   1               0.000
39051838  2024-02-06 23:00:00+00:00  TVB     Almacenamiento TVB  98307303   2          297308.000
          2024-02-19 23:00:00+00:0

#### Facturas

In [157]:
# TODO: usar columnas extras del Excel inicial de Facturas (validación BO, Conciliado, ...)

# hacemos lo mismo para facturas
facturas3_grouped = facturas_merged3_df.groupby(['NumeroFactura', 'FechaFactura', 'Origen', 'Servicio facturado'])[
    'Importe'].sum()

facturas3_grouped.head(100)

NumeroFactura  FechaFactura               Origen  Servicio facturado
2024162757     2024-02-02 00:00:00+00:00  TVB     Almacenamiento TVB    384552.96
2024166293     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB    457011.54
2024166294     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      4950.06
2024166295     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      4715.04
2024166296     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      4597.56
2024166297     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      4832.58
2024166298     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      4480.08
2024166299     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      1877.34
2024166300     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      4341.60
2024166301     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      1759.92
2024166302     2024-02-04 00:00:00+00:00  TVB     Almacenamiento TVB      1407.78
2024166303     2024-02-04 00:

#### Dataframe de Conciliación

In [159]:
# creamos un dataframe solo con el importe total de los deals
CTV_new_final_df = CTV_new_merged_df.groupby(['Id Deal','Origen', 'Servicio facturado'])['Importe Flow'].sum()
CTV_new_final_df.head(100)

Id Deal   Origen  Servicio facturado
34835329  TVB     Almacenamiento TVB         0.000
37734719  TVB     Almacenamiento TVB         0.000
37737643  TVB     Almacenamiento TVB         0.000
37761994  TVB     Almacenamiento TVB         0.000
39051838  TVB     Almacenamiento TVB    594616.000
39057923  TVB     Almacenamiento TVB         0.002
39942100  TVB     Almacenamiento TVB         0.000
39980344  TVB     Almacenamiento TVB     64092.160
Name: Importe Flow, dtype: float64

In [158]:
# unimos los dataframes para comparar
conciliacion_new_df = pd.merge(CTV_new_final_df, facturas3_grouped, on=['Origen', 'Servicio facturado'], how='outer')

# hallamos la diferencia 
conciliacion_new_df['Factura-Deal'] = conciliacion_new_df['Importe'] - conciliacion_new_df['Importe Flow']  

conciliacion_new_df.head()

Importe Flow    Importe  Factura-Deal
Origen Servicio facturado                                       
TVB    Almacenamiento TVB           0.0  384552.96     384552.96
       Almacenamiento TVB           0.0  457011.54     457011.54
       Almacenamiento TVB           0.0    4950.06       4950.06
       Almacenamiento TVB           0.0    4715.04       4715.04
       Almacenamiento TVB           0.0    4597.56       4597.56